# Задание

Попробуйте настроить параметры градиентного бустинга на этом датасете, и посмотрите, как изменение отдельных параметров влияет на результат.

#### Конечная цель - сказать значение параметров xgb, при которых будет самое хорошее качество на кроссвалидации

In [9]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import pandas
import numpy as np
import signal
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

bioresponce = pandas.read_csv('../Seminar 1/bioresponse.csv', header=0, sep=',')
bioresponce_target = bioresponce.Activity.values
bioresponce_data = bioresponce.iloc[:, 1:]

def signal_handler(signum, frame):
    raise Exception("Timed out!")

def estimate_params(params):
    # signal.signal(signal.SIGALRM, signal_handler)
    # signal.alarm(30)
    estimator = xgb.XGBClassifier(**params)

    try:
        score = np.mean(cross_val_score(
            estimator, bioresponce_data, bioresponce_target,
            scoring = 'accuracy', cv = 3
        ))
    except Exception:
        score = None
    
    return score

In [10]:
print( estimate_params({
    'learning_rate': 0.1, 
    'max_depth': 5, 
    'n_estimators': 100, 
    'min_child_weight': 3,
    'seed': 42
}))

0.791254356515


In [11]:
print( estimate_params({
    'learning_rate': 0.1, 
    'max_depth': 5, 
    'n_estimators': 1000, 
    'min_child_weight': 3,
    'seed': 42
}))

0.792053503864


In [12]:
param_grid = {'learning_rate': np.arange(0, 0.5, 0.1), 'max_depth' : np.arange(5, 8, 1), 
              'n_estimators': np.arange(1000, 1500, 500), 'min_child_weight' : np.arange(1, 4, 1)}
estimator = RandomizedSearchCV(xgb.XGBClassifier(seed=42), param_grid, cv=5).fit(bioresponce_data, bioresponce_target)
best_param = estimator.best_params_
print( estimate_params(best_param))

0.79178705036


In [14]:
print( estimate_params({
    'learning_rate': 0.1, 
    'max_depth': 7, 
    'n_estimators': 1000, 
    'min_child_weight': 1,
    'seed': 42
}))

0.796585771383
